**예제 1. Generator 구성하기**

**예제 2. Discriminator 구성하기**

**예제 3. 구성한 Generator 와 Discriminator 를 학습해 새로운 MNIST 이미지 생성**

*   필요한 Library Import

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from IPython import display

from PIL import Image
from torch import autograd
from torch.autograd import Variable
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
from torchvision import transforms, datasets

*   학습에 사용될 Hyper Parameter 설정

In [ ]:
channel = 3
generator_filter = 64 
discriminator_filter = 64
epochs = 200
lr = 0.0002
beta1 = 0.5
epochs = 10

image_size = 64
batch_size = 64
output_path = "out"

*   학습에 필요한 CIFAR-10 데이터셋 다운로드

In [ ]:
transform = transforms.Compose([
        transforms.Resize([64, 64]), 
        transforms.ToTensor(),                     
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

cifar10_dataset = datasets.CIFAR10(root='./data/', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(cifar10_dataset, batch_size= batch_size, shuffle=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data/


*   weight 초기화

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:         # Conv weight init
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:  # BatchNorm weight init
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

*   Generator 구성

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, generator_filter * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(generator_filter * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(generator_filter * 8, generator_filter * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(generator_filter*4),
            nn.ReLU(True),
            nn.ConvTranspose2d(generator_filter * 4, generator_filter * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(generator_filter*2),
            nn.ReLU(True),
            nn.ConvTranspose2d(generator_filter * 2, generator_filter, 4, 2, 1, bias=False),
            nn.BatchNorm2d(generator_filter),
            nn.ReLU(True),
            nn.ConvTranspose2d(generator_filter, channel, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        output = self.main(input)
        return output

*   Discriminator 구성

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(channel, discriminator_filter, 4,2,1,bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(discriminator_filter, discriminator_filter * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(discriminator_filter * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(discriminator_filter*2, discriminator_filter*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(discriminator_filter * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(discriminator_filter * 4, discriminator_filter * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(discriminator_filter*8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(discriminator_filter*8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.main(input)
        return output.view(-1, 1).squeeze(1)

*   Loss Function 과 Optimization 정의

In [ ]:
generator = Generator()
generator.apply(weights_init)

discriminator = Discriminator()
discriminator.apply(weights_init)

criterion = nn.BCELoss()

input = torch.FloatTensor(batch_size, 3, image_size, image_size)
noise = torch.FloatTensor(batch_size, 100, 1, 1)
noise = torch.FloatTensor(batch_size, 100, 1, 1).normal_(0, 1)
noise = Variable(noise)

label = torch.FloatTensor(batch_size)
real_label = 1
fake_label = 0

In [ ]:
optimizerG = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerD = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))

*   학습

In [ ]:
for epoch in range(epochs):
    for i, data in enumerate(train_loader):
        discriminator.zero_grad()
        image, _ = data
        batch_size = image.size(0)

        input.resize_as_(image).copy_(image)
        label.resize_(batch_size).fill_(real_label)

        input_var = Variable(input)
        label_var = Variable(label)

        output = discriminator(input_var)
        loss_D_real = criterion(output, label_var)
        loss_D_real.backward()
        real_score = output.data.mean()

        noise.resize_(batch_size, 100, 1, 1).normal_(0, 1)
        noise_var = Variable(noise)
        fake = generator(noise_var)
        label_var = Variable(label.fill_(fake_label))
        output = discriminator(fake.detach())
        loss_D_fake = criterion(output, label_var)
        loss_D_fake.backward()
        fake_score = output.data.mean()

        loss_D = loss_D_real + loss_D_fake
        optimizerD.step()

        generator.zero_grad()
        label_var = Variable(label.fill_(real_label))
        output = discriminator(fake)

        loss_G = criterion(output, label_var)
        loss_G.backward()
        G_score = output.data.mean()
        optimizerG.step()
        result_dict = {"loss_D":loss_D,"loss_G":loss_G,"real_score":real_score,"fake_score":fake_score,"loss_G":loss_G}

*   학습결과 확인

In [ ]:
fake_image = Generator(noise)

for i in range(5):
    fake_out = np.reshape(fake_image.data.cpu().numpy()[i],(28, 28))
    plt.imshow(fake_out, cmap = 'gray')
    plt.show()